In [1]:
from datasets import list_datasets, load_dataset, list_metrics, load_metric

In [4]:
squad_dataset = load_dataset('squad')

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/4c81550d83a2ac7c7ce23783bd8ff36642800e6633c1f18417fb58c3ff50cdd7. Subsequent calls will reuse this data.


In [12]:
squad_dataset["train"]["answers"]

[{'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
 {'answer_start': [188], 'text': ['a copper statue of Christ']},
 {'answer_start': [279], 'text': ['the Main Building']},
 {'answer_start': [381], 'text': ['a Marian place of prayer and reflection']},
 {'answer_start': [92], 'text': ['a golden statue of the Virgin Mary']},
 {'answer_start': [248], 'text': ['September 1876']},
 {'answer_start': [441], 'text': ['twice']},
 {'answer_start': [598], 'text': ['The Observer']},
 {'answer_start': [126], 'text': ['three']},
 {'answer_start': [908], 'text': ['1987']},
 {'answer_start': [119], 'text': ['Rome']},
 {'answer_start': [145], 'text': ['Moreau Seminary']},
 {'answer_start': [234], 'text': ['Old College']},
 {'answer_start': [356], 'text': ['Retired priests and brothers']},
 {'answer_start': [675], 'text': ['Buechner Prize for Preaching']},
 {'answer_start': [487], 'text': ['eight']},
 {'answer_start': [46], 'text': ['1920']},
 {'answer_start': [126], 'text': ['the College

In [8]:
squad_dataset.map(lambda x: {"length": len(x["context"])})

Loading cached processed dataset at /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/4c81550d83a2ac7c7ce23783bd8ff36642800e6633c1f18417fb58c3ff50cdd7/cache-ff73dcd1c8abea1f.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/4c81550d83a2ac7c7ce23783bd8ff36642800e6633c1f18417fb58c3ff50cdd7/cache-995cd76e77ca6b01.arrow


DatasetDict({
    train: Dataset({
        features: ['answers', 'context', 'id', 'length', 'question', 'title'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['answers', 'context', 'id', 'length', 'question', 'title'],
        num_rows: 10570
    })
})

NameError: name 'wikipedia_cbor' is not defined

In [9]:
#from transformers import AutoTokenizer
#'tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

print(list_metrics())

['accuracy', 'bertscore', 'bleu', 'bleurt', 'coval', 'f1', 'gleu', 'glue', 'indic_glue', 'meteor', 'precision', 'recall', 'rouge', 'sacrebleu', 'seqeval', 'squad', 'squad_v2', 'xnli']


In [10]:
squad_dataset["train"]["answers"][:5]

[{'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
 {'answer_start': [188], 'text': ['a copper statue of Christ']},
 {'answer_start': [279], 'text': ['the Main Building']},
 {'answer_start': [381], 'text': ['a Marian place of prayer and reflection']},
 {'answer_start': [92], 'text': ['a golden statue of the Virgin Mary']}]

In [11]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening under the hood, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize input
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)

# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = 8
tokenized_text[masked_index] = '[MASK]'
assert tokenized_text == ['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', '[MASK]', 'was', 'a', 'puppet', '##eer', '[SEP]']

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])



INFO:filelock:Lock 140712192193816 acquired on /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock


INFO:filelock:Lock 140712192193816 released on /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock


In [12]:
# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cuda')
segments_tensors = segments_tensors.to('cuda')
model.to('cuda')

# Predict all tokens
with torch.no_grad():
    outputs = model(tokens_tensor, token_type_ids=segments_tensors)
    predictions = outputs[0]

# confirm we were able to predict 'henson'
predicted_index = torch.argmax(predictions[0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
assert predicted_token == 'henson'

INFO:filelock:Lock 140710216695200 acquired on /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.637c6035640bacb831febcc2b7f7bee0a96f9b30c2d7e9ef84082d9f252f3170.lock


INFO:filelock:Lock 140710216695200 released on /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.637c6035640bacb831febcc2b7f7bee0a96f9b30c2d7e9ef84082d9f252f3170.lock


INFO:filelock:Lock 140710216695200 acquired on /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f.lock


INFO:filelock:Lock 140710216695200 released on /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f.lock


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [32]:
# what happens with 2?

tokenized_text[masked_index+2] = '[MASK]'
assert tokenized_text == ['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', '[MASK]', 'was', '[MASK]', 'puppet', '##eer', '[SEP]']
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
tokens_tensor = torch.tensor([indexed_tokens]).to('cuda')

# Predict all tokens
with torch.no_grad():
    outputs = model(tokens_tensor, token_type_ids=segments_tensors, output_hidden_states=True)
    
    for hid in outputs.hidden_states:
        print(hid.size())
    predictions = outputs.logits[0]

torch.Size([1, 14, 768])
torch.Size([1, 14, 768])
torch.Size([1, 14, 768])
torch.Size([1, 14, 768])
torch.Size([1, 14, 768])
torch.Size([1, 14, 768])
torch.Size([1, 14, 768])
torch.Size([1, 14, 768])
torch.Size([1, 14, 768])
torch.Size([1, 14, 768])
torch.Size([1, 14, 768])
torch.Size([1, 14, 768])
torch.Size([1, 14, 768])


In [33]:
predictions

tensor([[[ -7.8907,  -7.8076,  -7.8029,  ...,  -7.0465,  -6.7833,  -4.5639],
         [-13.6117, -13.9999, -13.9970,  ..., -11.8237, -11.1671, -14.1446],
         [-11.4372, -11.0038, -11.2873,  ..., -11.7792,  -8.5248,  -6.4698],
         ...,
         [ -4.5049,  -5.0288,  -4.7041,  ...,  -2.7198,  -2.8374,  -6.6997],
         [-12.4046, -12.9720, -13.0765,  ...,  -9.6064,  -9.3818, -10.1639],
         [-12.8552, -12.5336, -12.6702,  ..., -10.1807, -10.0876,  -9.0030]]],
       device='cuda:0')

In [28]:
predicted_index_1 = torch.argmax(predictions[0, masked_index]).item()
predicted_index_2 = torch.argmax(predictions[0, masked_index+2]).item()

print(tokenizer.convert_ids_to_tokens([predicted_index_1, predicted_index_2]))

['henson', 'a']


In [16]:
import datasets

squad_metric, squad_v2_metric = datasets.load_metric('squad'), datasets.load_metric('squad_v2')

In [17]:
squad_metric

Metric(name: "squad", features: {'predictions': {'id': Value(dtype='string', id=None), 'prediction_text': Value(dtype='string', id=None)}, 'references': {'id': Value(dtype='string', id=None), 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}}, usage: """
Computes SQuAD scores (F1 and EM).
Args:
    predictions: List of question-answers dictionaries with the following key-values:
        - 'id': id of the question-answer pair as given in the references (see below)
        - 'prediction_text': the text of the answer
    references: List of question-answers dictionaries with the following key-values:
        - 'id': id of the question-answer pair (see above),
        - 'answers': a Dict in the SQuAD dataset format
            {
                'text': list of possible texts for the answer, as a list of strings
                'answer_start': list of start positions for the answer, as a list of ints
   

In [18]:
squad_v2_metric

Metric(name: "squad_v2", features: {'predictions': {'id': Value(dtype='string', id=None), 'prediction_text': Value(dtype='string', id=None), 'no_answer_probability': Value(dtype='float32', id=None)}, 'references': {'id': Value(dtype='string', id=None), 'answers': Sequence(feature={'text': Value(dtype='string', id=None), 'answer_start': Value(dtype='int32', id=None)}, length=-1, id=None)}}, usage: """
Computes SQuAD v2 scores (F1 and EM).
Args:
    predictions: List of triple for question-answers to score with the following elements:
        - the question-answer 'id' field as given in the references (see below)
        - the text of the answer
        - the probability that the question has no answer
    references: List of question-answers dictionaries with the following key-values:
            - 'id': id of the question-answer pair (see above),
            - 'answers': a list of Dict {'text': text of the answer as a string}
    no_answer_threshold: float
        Probability threshold

In [21]:
print(squad_metric.description)


This metric wrap the official scoring script for version 1 of the Stanford Question Answering Dataset (SQuAD).

Stanford Question Answering Dataset (SQuAD) is a reading comprehension dataset, consisting of questions posed by
crowdworkers on a set of Wikipedia articles, where the answer to every question is a segment of text, or span,
from the corresponding reading passage, or the question might be unanswerable.



In [24]:
from tools.vocabs import load_tokenizer

tokenizer = load_tokenizer('bert-base-uncased')

def get_prediction_as_str(identifider, input_ids, start_token, end_token):
    return {
        "id": identifier,
        "prediction_text": "".join(tokenizer.id_to_token(x) \
                                   for x in input_ids[start_token:end_token+1])
    }



In [28]:
from tools.dataloaders import SQuADDataloader
from torch.utils.data import DataLoader

squad_dataset = SQuADDataloader()
squad_train_dataset = squad_dataset.train_dataset
squad_train_dataloader = DataLoader(squad_train_dataset, batch_size=8)
squad_validation_dataset = squad_dataset.validation_dataset
squad_validation_dataloader = DataLoader(squad_validation_dataset, batch_size=8)

Reusing dataset squad (/root/.cache/huggingface/datasets/squad/plain_text/1.0.0/4c81550d83a2ac7c7ce23783bd8ff36642800e6633c1f18417fb58c3ff50cdd7)
Loading cached processed dataset at /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/4c81550d83a2ac7c7ce23783bd8ff36642800e6633c1f18417fb58c3ff50cdd7/cache-6f5b1d9507b91c14.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/4c81550d83a2ac7c7ce23783bd8ff36642800e6633c1f18417fb58c3ff50cdd7/cache-94ee633b92a1c5d3.arrow


In [29]:


for b in squad_validation_dataloader:
    print(b)
    break

{'answer_end': tensor([ 35,  46,  83,  35,  98, 100,  66,  29]), 'answer_start': tensor([34, 45, 80, 34, 98, 97, 63, 27]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'input_ids': tensor([[ 101, 3565, 4605,  ...,    0,    0,    0],
        [ 101, 3565, 4605,  ...,    0,    0,    0],
        [ 101, 3565, 4605,  ...,    0,    0,    0],
        ...,
        [ 101, 3565, 4605,  ...,    0,    0,    0],
        [ 101, 3565, 4605,  ...,    0,    0,    0],
        [ 101, 3565, 4605,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])}
